### Importación de los módulos necesarios

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pmdarima.arima import ndiffs, nsdiffs
from datetime import datetime
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
from dateutil.relativedelta import relativedelta
from metricas import calculo_metricas

import warnings
warnings.simplefilter("ignore")

# Mostramos las versiones de los módulos para posibles reproducciones del código

print('Versión pandas:', pd.__version__)
print('Versión numpy:', np.__version__)
print('Versión matplotlib:', matplotlib.__version__)
print('Version statsmodels', sm.__version__)

Versión pandas: 1.0.5
Versión numpy: 1.19.5
Versión matplotlib: 3.2.2
Version statsmodels 0.12.2


### Lectura del dataframe horario

In [2]:
df = pd.read_csv('Data/datos_horarios_prediccion_electricidad.csv', index_col = 0)

df.index = pd.to_datetime(df.index)

df = df.sort_index()


In [3]:
df = pd.read_csv('Data/dataframe.csv', index_col = 0)
df.index = pd.to_datetime(df.index)
df = df.drop(columns = ["Festivo_Regional", "Humedad_Relativa", "Precipitacion", "Radiacion", "Velocidad_Viento"])

df.loc[:, "lag_24"] = df.Spot_electricidad.shift(24)
df.loc[:, "lag_48"] = df.Spot_electricidad.shift(48)
df.loc[:, "lag_1_semana"] = df.Spot_electricidad.shift(24*7)

### Modelo regresión lineal

In [33]:
db = df.dropna()
# db = db.drop(columns = ["Prevision_Demanda", "Prevision_Eol_Fotov", "Festivo_Regional", "Brent",
#                        "Humedad_Relativa", "Precipitacion", "Radiacion"])
db = db.drop(columns = ["Prevision_Demanda", "Prevision_Eol_Fotov", "Festivo_Regional", "Brent",
                       "Humedad_Relativa", "Precipitacion", "Radiacion", "Velocidad_Viento"])



db = db.dropna()
X = sm.add_constant(db)
y = X.pop('Spot_electricidad')

model = sm.OLS(y,X).fit()

model.summary()

### Backtesting regresión lineal

In [7]:
def bascktesting_regresion_lineal(db, año, mes, rolling_window, days_start):
    fecha = datetime(año, mes, 1)
    fecha_str = str(año) + '-' + str(mes) + '-' + str(1)
    
    db = db.dropna()
    array_real, array_pred, array_date = np.zeros(len(db.loc[fecha_str:])), np.zeros(len(db.loc[fecha_str:])), []

    # En caso de que haya rolling window
    if rolling_window == True:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_start_train = fecha_end_train - relativedelta(days = days_start)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            fecha_start_train = str(fecha_start_train.year) + "-" + str(fecha_start_train.month) + "-" + str(fecha_start_train.day)
            db_aux = db.loc[fecha_start_train:fecha_end_train]
            
            # Dataframe de Entrenamiento
            X_train = sm.add_constant(db_aux)
            y_train = X_train.pop('Spot_electricidad')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)

            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)

            # Entrenamos el modelo
            model = sm.OLS(y_train, X_train).fit()
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = pred.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
        
    # En caso de que no haya rolling window, siempre cogemos todo el dataframe disponible
    else:
        for day in range(int(len(db.loc[fecha_str:]) / 24)):
            fecha_end_train = fecha + relativedelta(days = day)
            fecha_end_train = str(fecha_end_train.year) + "-" + str(fecha_end_train.month) + "-" + str(fecha_end_train.day)
            db_aux = db.loc[:fecha_end_train]

            # Dataframe de Entrenamiento
            X_train = sm.add_constant(db_aux)
            y_train = X_train.pop('Spot_electricidad')
            
            # Dataframe de Test
            X_test = X_train.loc[fecha_end_train]
            y_test = y_train.pop(fecha_end_train)
            
            # Quitamos al dataframe de train, el dataframe de test
            X_train = X_train.drop(index = X_train.iloc[-24:].index)
            
            # Entrenamos el modelo
            model = sm.OLS(y_train, X_train).fit()
            
            # Realizamos las predicciones
            pred = model.predict(X_test)
            date = pred.index

            array_date.append(date)
            array_pred[day*24:day*24 + 24] = pred
            array_real[day*24:day*24 + 24] = y_test
            
    array_date = [item for sublist in array_date for item in sublist]
    return array_pred, array_real, array_date
    
pred, real, dates = bascktesting_regresion_lineal(db.loc["2019":], 2020, 6, rolling_window = False, days_start = 30)

In [9]:
# Backtesting rapido

for dias in [7, 14, 21, 30, 60, 90, 150, 220, 365]:
    print(dias)
    pred, real, dates = bascktesting_regresion_lineal(db.loc["2019":], 2020, 6, rolling_window = True, days_start = dias)
    resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
    resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0
    calculo_metricas(resultados)

7
     MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  20.53          9.57  151.78  18.35  46.43    81.43
14
      MAE  MAE (median)    MAPE   WMAPE     RMSE  % Trend
0  166.02         23.73  179.51  148.32  1682.29    80.95
21
     MAE  MAE (median)   MAPE  WMAPE   RMSE  % Trend
0  14.23          9.46  91.96  12.71  21.62    83.39
30
     MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  11.95          8.12  105.58  10.68  18.82    83.58
60
     MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  10.62          7.12  123.34   9.49  16.82     84.2
90
     MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  10.58          7.31  141.23   9.45  16.59    84.15
150
     MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  10.49          6.96  135.08   9.37  16.72    84.24
220
     MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  10.47          7.05  133.98   9.35  16.79    84.35
365
     MAE  MAE (median)    MAPE  WMAPE   RMSE  % Trend
0  11.22          7.39  127.68  10.02  18.49   

### Cálculo de las métricas

In [8]:
resultados = pd.DataFrame({"Pred":pred, "Real":real}, index = dates)
resultados["Pred"][resultados.loc[:, "Pred"] < 0] = 0


reg_lineal, reg_lineal_metricas_mensuales, met = calculo_metricas(resultados)

    MAE  MAE (median)    MAPE  WMAPE  RMSE  % Trend
0  12.0          7.75  153.66  10.72  20.2    83.94


In [ ]:
reg_lineal.to_csv('Resultados/resultados_reg_lineal.csv')

In [ ]:
import plotly.graph_objects as go

def graficas(dataframe):
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dataframe.loc["2019-01":].index, y=dataframe.loc["2019-01":].Real,
                        mode='lines',
                        name='Real')).add_trace(go.Scatter(x=dataframe.loc["2019-01":].index, y=dataframe.loc["2019-01":].Pred,
                        mode='lines',
                        name='Predicciones'))
    
    fig.show()
    
    # Create traces
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=dataframe.loc["2019-07"].index, y=dataframe.loc["2019-07"].Real,
#                     mode='lines',
#                     name='Real')).add_trace(go.Scatter(x=dataframe.loc["2019-07"].index, y=dataframe.loc["2019-07"].Pred,
#                     mode='lines',
#                     name='Predicciones'))

#     fig.show()
    
graficas(reg_lineal)